# KNN (K-Nearest-Neighbors)

# Atividade no final do arquivo

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [1]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('../dependencias/arquivos_de_codigo/u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

/tmp/ipykernel_29509/2628946610.py:3: FutureWarning: The provided callable <function mean at 0x7002141123e0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})


rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [4]:
movieDict = {}
with open(r'../dependencias/arquivos_de_codigo/u.item', encoding="ISO-8859-1") as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


For example, here's the record we end up with for movie ID 1, "Toy Story":

In [5]:
print(movieDict[1])

('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.7735849056603774, 3.8783185840707963)


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [6]:
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    
ComputeDistance(movieDict[2], movieDict[4])



0.8004574042309892

Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

In [7]:
print(movieDict[2])
print(movieDict[4])


('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.3567753001715266, 3.550239234449761)


Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:

In [8]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [9]:
avgRating

3.3445905900235564

How does this compare to Toy Story's actual average rating?

In [10]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

Not too bad!


## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

#### Testando valores diferentes de K:

In [17]:
neighbors3 = getNeighbors(1, 3)
neighbors5 = getNeighbors(1, 5)
neighbors15 = getNeighbors(1, 15)
neighbors20 = getNeighbors(1, 20)
neighbors25 = getNeighbors(1, 25)
neighbors30 = getNeighbors(1, 30)

avgRating3 = 0
for neighbor in neighbors3:
    avgRating3 += movieDict[neighbor][3] / 5

avgRating5 = 0
for neighbor in neighbors5:
    avgRating5 += movieDict[neighbor][3] / 5

avgRating15 = 0
for neighbor in neighbors15:
    avgRating15 += movieDict[neighbor][3] / 15

avgRating20 = 0
for neighbor in neighbors20:
    avgRating20 += movieDict[neighbor][3] / 20

avgRating25 = 0
for neighbor in neighbors25:
    avgRating25 += movieDict[neighbor][3] / 25

avgRating30 = 0
for neighbor in neighbors30:
    avgRating30 += movieDict[neighbor][3] / 30

print(f'A nota, de fato: {movieDict[1][3]}')
print(f'Para K = 3, a previsão é: {avgRating3}')
print(f'Para K = 5, a previsão é: {avgRating5}')
print(f'Para K = 15, a previsão é: {avgRating15}')
print(f'Para K = 20, a previsão é: {avgRating20}')
print(f'Para K = 25, a previsão é: {avgRating25}')
print(f'Para K = 30, a previsão é: {avgRating30}')

A nota, de fato: 3.8783185840707963
Para K = 3, a previsão é: 2.120277651909297
Para K = 5, a previsão é: 3.7189656165466287
Para K = 15, a previsão é: 3.4669102337544855
Para K = 20, a previsão é: 3.499848386860249
Para K = 25, a previsão é: 3.508639129932606
Para K = 30, a previsão é: 3.4941020557018505


Podemos observar que, neste caso, o valor de K que gerou a melhor previsão foi o de 5, menos que isso se afasta bastante do valor real, e conforme aumenta, a previsão tende a piorar estagnando em torno de 3.5.

#### Alterando a métrica de distância:

In [18]:
def ComputeDistance2(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return 0.6*genreDistance + 0.4*popularityDistance
#nesta versão da ComputeDistance, 60% do peso da distância final é dada pela diferença entre os gêneros

def getNeighborsAlt(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance2(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

neighbors3 = getNeighborsAlt(1, 3)
neighbors5 = getNeighborsAlt(1, 5)
neighbors15 = getNeighborsAlt(1, 15)
neighbors20 = getNeighborsAlt(1, 20)
neighbors25 = getNeighborsAlt(1, 25)
neighbors30 = getNeighborsAlt(1, 30)

avgRating3 = 0
for neighbor in neighbors3:
    avgRating3 += movieDict[neighbor][3] / 5

avgRating5 = 0
for neighbor in neighbors5:
    avgRating5 += movieDict[neighbor][3] / 5

avgRating15 = 0
for neighbor in neighbors15:
    avgRating15 += movieDict[neighbor][3] / 15

avgRating20 = 0
for neighbor in neighbors20:
    avgRating20 += movieDict[neighbor][3] / 20

avgRating25 = 0
for neighbor in neighbors25:
    avgRating25 += movieDict[neighbor][3] / 25

avgRating30 = 0
for neighbor in neighbors30:
    avgRating30 += movieDict[neighbor][3] / 30

print(f'A nota, de fato: {movieDict[1][3]}')
print(f'Para K = 3, a previsão é: {avgRating3}')
print(f'Para K = 5, a previsão é: {avgRating5}')
print(f'Para K = 15, a previsão é: {avgRating15}')
print(f'Para K = 20, a previsão é: {avgRating20}')
print(f'Para K = 25, a previsão é: {avgRating25}')
print(f'Para K = 30, a previsão é: {avgRating30}')

A nota, de fato: 3.8783185840707963
Para K = 3, a previsão é: 2.120277651909297
Para K = 5, a previsão é: 3.226545458177103
Para K = 15, a previsão é: 3.365438948852873
Para K = 20, a previsão é: 3.4073766648933024
Para K = 25, a previsão é: 3.394696305753645
Para K = 30, a previsão é: 3.361800970062235


Podemos observar que as previsões pioraram um pouco com a mudança na ComputeDistance.

In [20]:
def ComputeDistance3(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return 0.4*genreDistance + 0.6*popularityDistance
#nesta versão da ComputeDistance, 60% do peso da distância final é dada pela diferença entre as popularidades

def getNeighborsAlt(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance3(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

neighbors3 = getNeighborsAlt(1, 3)
neighbors5 = getNeighborsAlt(1, 5)
neighbors15 = getNeighborsAlt(1, 15)
neighbors20 = getNeighborsAlt(1, 20)
neighbors25 = getNeighborsAlt(1, 25)
neighbors30 = getNeighborsAlt(1, 30)

avgRating3 = 0
for neighbor in neighbors3:
    avgRating3 += movieDict[neighbor][3] / 5

avgRating5 = 0
for neighbor in neighbors5:
    avgRating5 += movieDict[neighbor][3] / 5

avgRating15 = 0
for neighbor in neighbors15:
    avgRating15 += movieDict[neighbor][3] / 15

avgRating20 = 0
for neighbor in neighbors20:
    avgRating20 += movieDict[neighbor][3] / 20

avgRating25 = 0
for neighbor in neighbors25:
    avgRating25 += movieDict[neighbor][3] / 25

avgRating30 = 0
for neighbor in neighbors30:
    avgRating30 += movieDict[neighbor][3] / 30

print(f'A nota, de fato: {movieDict[1][3]}')
print(f'Para K = 3, a previsão é: {avgRating3}')
print(f'Para K = 5, a previsão é: {avgRating5}')
print(f'Para K = 15, a previsão é: {avgRating15}')
print(f'Para K = 20, a previsão é: {avgRating20}')
print(f'Para K = 25, a previsão é: {avgRating25}')
print(f'Para K = 30, a previsão é: {avgRating30}')

A nota, de fato: 3.8783185840707963
Para K = 3, a previsão é: 2.120277651909297
Para K = 5, a previsão é: 3.7189656165466287
Para K = 15, a previsão é: 3.5920367089657566
Para K = 20, a previsão é: 3.5467874455572637
Para K = 25, a previsão é: 3.631153059258684
Para K = 30, a previsão é: 3.589437299981924


Nesta versão, a melhor previsão (K = 5) e as com o K menor que 5 permaneceram muito parecidas com quando a ComputeDistance original é usada, porém as com o K > 5 ficaram um pouco melhores.

In [21]:
def ComputeDistance2(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return 0.3*genreDistance + 0.7*popularityDistance
#nesta versão da ComputeDistance, 30% do peso da distância final é dada pela diferença entre os gêneros

def getNeighborsAlt(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance2(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

neighbors3 = getNeighborsAlt(1, 3)
neighbors5 = getNeighborsAlt(1, 5)
neighbors15 = getNeighborsAlt(1, 15)
neighbors20 = getNeighborsAlt(1, 20)
neighbors25 = getNeighborsAlt(1, 25)
neighbors30 = getNeighborsAlt(1, 30)

avgRating3 = 0
for neighbor in neighbors3:
    avgRating3 += movieDict[neighbor][3] / 5

avgRating5 = 0
for neighbor in neighbors5:
    avgRating5 += movieDict[neighbor][3] / 5

avgRating15 = 0
for neighbor in neighbors15:
    avgRating15 += movieDict[neighbor][3] / 15

avgRating20 = 0
for neighbor in neighbors20:
    avgRating20 += movieDict[neighbor][3] / 20

avgRating25 = 0
for neighbor in neighbors25:
    avgRating25 += movieDict[neighbor][3] / 25

avgRating30 = 0
for neighbor in neighbors30:
    avgRating30 += movieDict[neighbor][3] / 30

print(f'A nota, de fato: {movieDict[1][3]}')
print(f'Para K = 3, a previsão é: {avgRating3}')
print(f'Para K = 5, a previsão é: {avgRating5}')
print(f'Para K = 15, a previsão é: {avgRating15}')
print(f'Para K = 20, a previsão é: {avgRating20}')
print(f'Para K = 25, a previsão é: {avgRating25}')
print(f'Para K = 30, a previsão é: {avgRating30}')

A nota, de fato: 3.8783185840707963
Para K = 3, a previsão é: 2.1710117135242326
Para K = 5, a previsão é: 3.7232656817782006
Para K = 15, a previsão é: 3.7536508310350296
Para K = 20, a previsão é: 3.825272669021639
Para K = 25, a previsão é: 3.842850178742018
Para K = 30, a previsão é: 3.832696925112364


Dando ainda mais peso à diferença de popularidade em detrimento da diferença de gênero, as previsões para K > 5 ficaram consideravelmente melhores, enquanto para K <= 5 pouco mudou.

In [22]:
def ComputeDistance2(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return 0.8*genreDistance + 0.2*popularityDistance
#nesta versão da ComputeDistance, 80% do peso da distância final é dada pela diferença entre os gêneros

def getNeighborsAlt(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance2(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

neighbors3 = getNeighborsAlt(1, 3)
neighbors5 = getNeighborsAlt(1, 5)
neighbors15 = getNeighborsAlt(1, 15)
neighbors20 = getNeighborsAlt(1, 20)
neighbors25 = getNeighborsAlt(1, 25)
neighbors30 = getNeighborsAlt(1, 30)

avgRating3 = 0
for neighbor in neighbors3:
    avgRating3 += movieDict[neighbor][3] / 5

avgRating5 = 0
for neighbor in neighbors5:
    avgRating5 += movieDict[neighbor][3] / 5

avgRating15 = 0
for neighbor in neighbors15:
    avgRating15 += movieDict[neighbor][3] / 15

avgRating20 = 0
for neighbor in neighbors20:
    avgRating20 += movieDict[neighbor][3] / 20

avgRating25 = 0
for neighbor in neighbors25:
    avgRating25 += movieDict[neighbor][3] / 25

avgRating30 = 0
for neighbor in neighbors30:
    avgRating30 += movieDict[neighbor][3] / 30

print(f'A nota, de fato: {movieDict[1][3]}')
print(f'Para K = 3, a previsão é: {avgRating3}')
print(f'Para K = 5, a previsão é: {avgRating5}')
print(f'Para K = 15, a previsão é: {avgRating15}')
print(f'Para K = 20, a previsão é: {avgRating20}')
print(f'Para K = 25, a previsão é: {avgRating25}')
print(f'Para K = 30, a previsão é: {avgRating30}')

A nota, de fato: 3.8783185840707963
Para K = 3, a previsão é: 1.868824883893377
Para K = 5, a previsão é: 3.044035439760867
Para K = 15, a previsão é: 3.276707548714861
Para K = 20, a previsão é: 3.1621598053057767
Para K = 25, a previsão é: 3.0742315663289896
Para K = 30, a previsão é: 2.9750998097423995


Aumentando mais o peso da diferença de popularidade a precisão das previsões caíram muito para todos os valores de K.